# Dogbreed image classification task

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


This model presents a trained model to classify dogs from dogbreed image dataset.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [2]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.1/270.1 kB 3.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.3/83.3 kB 1.4 MB/s eta 0:00:00:00:01

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3
import os
from sagemaker.session import Session
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch
import matplotlib.pyplot as plt
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

## Dataset
133 different dogbreed classes.

In [ ]:
#TODO: Fetch and upload the data to AWS S3

# Command to download and unzip data
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
!unzip dogImages.zip

In [ ]:
session = sagemaker.Session()
bucket = session.default_bucket()
os.environ["DEFAULT_S3_BUCKET"] = bucket
!aws s3 sync ./dogImages s3://${DEFAULT_S3_BUCKET}/

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [5]:
##### TODO: Declare your HP ranges, metrics etc.
hyperparameter_ranges = {
    "lr": ContinuousParameter(0.001, 0.1),
    "epochs": CategoricalParameter([2, 3, 4, 5]),
}
objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "average test loss", "Regex": "Test set: Average Loss: ([0-9\\.]+)"}]

In [6]:
#TODO: Create estimators for your HPs

role = get_execution_role()

estimator = PyTorch(
    entry_point="hpo.py",
    role=role,
    py_version='py36',
    framework_version="1.8",
    instance_count=1,
    instance_type="ml.m5.large"
)# TODO: Your estimator here

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=4,
    max_parallel_jobs=4,
    objective_type=objective_type,
)# TODO: Your HP tuner here

In [9]:
# TODO: Fit your HP Tuner
os.environ['SM_CHANNEL_TRAINING']='s3://sagemaker-us-east-1-909528142112/dogImages/'
os.environ['SM_MODEL_DIR']='s3://sagemaker-us-east-1-909528142112/model/'
os.environ['SM_OUTPUT_DATA_DIR']='s3://sagemaker-us-east-1-909528142112/output/'
tuner.fit({"training": "s3://sagemaker-us-east-1-909528142112/dogImages/"}) # TODO: Remember to include your data channels

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [10]:
# TODO: Get the best estimators and the best HPs

best_estimator = tuner.best_estimator()#TODO

# #Get the hyperparameters of the best trained model
best_estimator.hyperparameters()



2022-08-12 13:39:57 Starting - Preparing the instances for training
2022-08-12 13:39:57 Downloading - Downloading input data
2022-08-12 13:39:57 Training - Training image download completed. Training in progress.
2022-08-12 13:39:57 Uploading - Uploading generated training model
2022-08-12 13:39:57 Completed - Training job completed


{'_tuning_objective_metric': '"average test loss"',
 'epochs': '"3"',
 'lr': '0.0013276630540790676',
 'sagemaker_container_log_level': '20',
 'sagemaker_estimator_class_name': '"PyTorch"',
 'sagemaker_estimator_module': '"sagemaker.pytorch.estimator"',
 'sagemaker_job_name': '"pytorch-training-2022-08-12-12-40-05-234"',
 'sagemaker_program': '"hpo.py"',
 'sagemaker_region': '"us-east-1"',
 'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-909528142112/pytorch-training-2022-08-12-12-40-05-234/source/sourcedir.tar.gz"'}

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [14]:
# TODO: Set up debugging and profiling rules and hooks
from sagemaker.debugger import Rule, ProfilerRule, rule_configs, DebuggerHookConfig
rules = [
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport())
]
from sagemaker.debugger import ProfilerConfig, FrameworkProfile

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=10)
)
hook_config = DebuggerHookConfig(
    hook_parameters={
        "train.save_interval": "1",
        "eval.save_interval": "1"
    }
)
hyperparameters = {
'epochs': 3,
 'lr': 0.0013276630540790676
}

In [ ]:
# TODO: Create and fit an estimator

estimator = PyTorch(
    entry_point="train_model.py",
    role=role,
    py_version='py36',
    framework_version="1.8",
    instance_count=1,
    instance_type="ml.m5.large",
    hyperparameters=hyperparameters,
    profiler_config=profiler_config,
    rules=rules
)# TODO: Your estimator here

estimator.fit({"training": "s3://sagemaker-us-east-1-909528142112/dogImages/"})


2022-08-12 15:07:52 Starting - Starting the training job...
2022-08-12 15:08:20 Starting - Preparing the instances for trainingLossNotDecreasing: InProgress
LowGPUUtilization: InProgress
ProfilerReport: InProgress
.........
2022-08-12 15:09:45 Downloading - Downloading input data...............
2022-08-12 15:12:21 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-08-12 15:12:16,550 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-08-12 15:12:16,554 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-08-12 15:12:16,568 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-08-12 15:12:16,576 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-08-12 15:12:17,088 sagemaker-training-toolkit INFO     No GPUs 

In [17]:
# TODO: Plot a debugging output.
plt.plot(trial.steps(mode=modes.TRAIN),
         list(trial.tensor('CrossEntropyLoss_output_0').values(mode=modes.TRAIN).values()))
plt.show()

NameError: name 'plt' is not defined

Error: NameError: name 'logger' is not defined. I checked train_model.py, and modified it to import logging.

In [ ]:
# TODO: Display the profiler output
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts
import IPython
session = boto3.session.Session()
region = session.region_name

training_job_name = estimator.latest_training_job.name
print(f"Training jobname: {training_job_name}")
print(f"Region: {region}")

tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader=None,
    select_dimensions=["CPU", "GPU"],
    select_events=["total"],
)
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
print(f"You will find the profiler report in {rule_output_path}")

! aws s3 ls {rule_output_path} --recursive
! aws s3 cp {rule_output_path} ./ --recursive

profiler_report_name = [
    rule["RuleConfigurationName"]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]

IPython.display.HTML(filename=profiler_report_name + "/profiler-output/profiler-report.html")

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint
from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import Predictor

deployment=estimator.deploy(initial_instance_count=1, instance_type='ml.t2.medium') # TODO: Add your deployment configuration like instance type and number of instances
predictor = sagemaker.predictor.Predictor(
    deployment.endpoint_name,
    sagemaker_session=sagemaker.Session()
) ## TODO: fill in

In [ ]:
# TODO: Run an prediction on the endpoint
import torchvision.transforms as transforms

predictor.serializer = IdentitySerializer("image/png")
data_dir = './dogImages/test/002.Afghan_hound/Afghan_hound_hound_00116.jpg'
image = Image.open(data_dir)# TODO: Your code to load and preprocess image to send to endpoint for prediction
prepocessing = transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor()
        ])
image = preprocess(image)

response = predictor.predict(image, initial_args={"ContentType": "image/jpeg"})

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()